# Fjerde ordens Runge-Kutta metode for drevet harmonisk pendel
## Numerisk prosjekt vår 2025
Håkon Støren og Vetle Støren

Som gitt i oppgaven har vi at 
\begin{equation*}
    \frac{\textrm{d}^2\theta}{\textrm{d}t^2} + q\frac{\textrm{d}\theta}{\textrm{d}t}+\frac{g}{l}\sin{(\theta)}=\frac{F_D}{ml}\sin{(\omega_D t)},
\end{equation*}
hvor
- $\theta$ er utslagsvinkelen til pendelen,
- $g$ er tyngdeakselerasjonen,
- $l$ er lengden av snora,
- $q$ er en friksjonsparamter,
- $\frac{F_D}{ml}$ er drivkraften som pendelen blir utsatt for,
- $\omega_D$ er drivkraftens vinkelfrekvens. \
Vi har også fra oppgaveteksten at
- $\omega_0 = \sqrt{g/l}$ og er pendelens vinkelfrekvens

Vi antar at $\sin{\theta}\approx\theta$

In [1]:
# Importerer alle bibliotekene vi trenger
import numpy as np
import matplotlib.pyplot as plt
import scipy as sci

In [2]:
# Initialiserer alle konstanter
l = 1                   # m, lengden av snora
g = 9.8                 # m/s^2, tyngdeakselerasjon
theta_0 = 0.2           # rad, initiell vinkel med hensyn til vertikalen
theta_dot_0 = 0         # rad/s, initiell vinkelhastighet
q = 1.0                 # s^-1, friksjonsparameter
w_D = 3.13              # rad/s, drivkraftens vinkelfrekvens
drivkraft = 0.2         # s^-2, F_D/(ml)
